<a href="https://colab.research.google.com/github/livinNector/deep-learning-tools-lab/blob/main/5%20-%20Age%20Prediction%20Using%202D%20CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 5 - Age Prediction Using 2D CNN

## Importing Modules

In [1]:
import tensorflow as tf
import tensorflow.keras.layers as tfl
import os

## Dataset

- Dataset Used : WIKI face dataset from imdb-wiki
- Dataset Home Page : https://data.vision.ee.ethz.ch/cvl/rrothe/imdb-wiki/
- Dataset Url : https://data.vision.ee.ethz.ch/cvl/rrothe/imdb-wiki/static/wiki.tar.gz

In [2]:
url = "https://data.vision.ee.ethz.ch/cvl/rrothe/imdb-wiki/static/wiki_crop.tar"

dataset = tf.keras.utils.get_file(
    "wiki_crop", url,
    untar=True, cache_dir='.',
    cache_subdir=''
)
dataset_dir = os.path.join(os.path.dirname(dataset), 'wiki_crop')

### Loading and extracting the age from the meta data file

In [3]:
import scipy.io
# extract data from wiki.mat
mat = scipy.io.loadmat(os.path.join(dataset_dir,'wiki.mat'))

In [4]:
import numpy as np
import datetime

In [5]:
mat

{'__globals__': [],
 '__header__': b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Sat Jan 16 16:25:20 2016',
 '__version__': '1.0',
 'wiki': array([[(array([[723671, 703186, 711677, ..., 720620, 723893, 713846]], dtype=int32), array([[2009, 1964, 2008, ..., 2013, 2011, 2008]], dtype=uint16), array([[array(['17/10000217_1981-05-05_2009.jpg'], dtype='<U31'),
                 array(['48/10000548_1925-04-04_1964.jpg'], dtype='<U31'),
                 array(['12/100012_1948-07-03_2008.jpg'], dtype='<U29'), ...,
                 array(['09/9998109_1972-12-27_2013.jpg'], dtype='<U30'),
                 array(['00/9999400_1981-12-13_2011.jpg'], dtype='<U30'),
                 array(['80/999980_1954-06-11_2008.jpg'], dtype='<U29')]],
               dtype=object), array([[1., 1., 1., ..., 1., 1., 0.]]), array([[array(['Sami Jauhojärvi'], dtype='<U15'),
                 array(['Dettmar Cramer'], dtype='<U14'),
                 array(['Marc Okrand'], dtype='<U11'), ...,
                 arr

In [6]:
mat["wiki"]["dob"][0][0][0]

array([723671, 703186, 711677, ..., 720620, 723893, 713846], dtype=int32)

In [7]:
dob = np.vectorize(lambda x: datetime.datetime.fromordinal(x).year)(
    mat["wiki"]["dob"][0][0][0]
)
photo_taken = mat["wiki"]["photo_taken"][0][0][0]

In [8]:
age = (photo_taken-dob).astype(np.float32)

In [9]:
age

array([27., 38., 59., ..., 40., 29., 53.], dtype=float32)

In [10]:
mat["wiki"]["full_path"][0][0][0]

array([array(['17/10000217_1981-05-05_2009.jpg'], dtype='<U31'),
       array(['48/10000548_1925-04-04_1964.jpg'], dtype='<U31'),
       array(['12/100012_1948-07-03_2008.jpg'], dtype='<U29'), ...,
       array(['09/9998109_1972-12-27_2013.jpg'], dtype='<U30'),
       array(['00/9999400_1981-12-13_2011.jpg'], dtype='<U30'),
       array(['80/999980_1954-06-11_2008.jpg'], dtype='<U29')],
      dtype=object)

In [11]:
file_path = np.vectorize(lambda x : os.path.join(dataset_dir,x[0]))(
    mat["wiki"]["full_path"][0][0][0]
)

In [12]:
file_path

array(['./wiki_crop/17/10000217_1981-05-05_2009.jpg',
       './wiki_crop/48/10000548_1925-04-04_1964.jpg',
       './wiki_crop/12/100012_1948-07-03_2008.jpg', ...,
       './wiki_crop/09/9998109_1972-12-27_2013.jpg',
       './wiki_crop/00/9999400_1981-12-13_2011.jpg',
       './wiki_crop/80/999980_1954-06-11_2008.jpg'], dtype='<U49')

In [13]:
file_age_ds = tf.data.Dataset.from_tensor_slices((file_path,age))

In [14]:
def parse_function(filename, label):
    image_string = tf.io.read_file(filename)
    image_decoded = tf.io.decode_jpeg(image_string,channels=1)
    image = tf.image.resize(image_decoded, [256, 256])
    return image, tf.expand_dims(label,0)

In [15]:
image_age_ds = file_age_ds.map(parse_function).shuffle(seed=2,buffer_size=64)

In [16]:
image_age_ds

<ShuffleDataset element_spec=(TensorSpec(shape=(256, 256, 1), dtype=tf.float32, name=None), TensorSpec(shape=(1,), dtype=tf.float32, name=None))>

In [17]:
dataset_size = image_age_ds.cardinality().numpy()

In [19]:
AUTOTUNE = tf.data.AUTOTUNE
train_ds = image_age_ds.take(dataset_size*.6).batch(32).prefetch(AUTOTUNE)
val_ds = image_age_ds.skip(dataset_size*.6).take(dataset_size*.2).batch(32).prefetch(AUTOTUNE)
test_ds = image_age_ds.skip(dataset_size*.8).take(dataset_size*.2).batch(32).prefetch(AUTOTUNE)

## Creating the model

In [20]:
model = tf.keras.Sequential([
    tfl.Conv2D(32,(7,7),padding="valid",activation="relu",input_shape=(256,256,1)),
    tfl.MaxPool2D((4,4),strides = 4),
    tfl.Conv2D(64,(3,3),padding = "valid",activation="relu"),
    tfl.MaxPool2D((4,4),strides = 4),
    tfl.Conv2D(128,(3,3),padding = "valid",activation="relu"),
    tfl.MaxPool2D((2,2),strides = 2),
    tfl.Conv2D(256,(1,1),padding= "valid",activation="relu",),
    tfl.MaxPool2D((2,2),strides = 2),
    tfl.Flatten(),
    tfl.Dense(64,activation="relu"),
    tfl.Dense(1)                             
])

In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 250, 250, 32)      1600      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 62, 62, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 60, 60, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 15, 15, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 13, 13, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 6, 6, 128)        0

In [22]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.MeanAbsoluteError(),
    metrics=['MAE']
)

In [ ]:
model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,
    callbacks=[
        tf.keras.callbacks.TensorBoard(log_dir="logs")
    ]
)

Epoch 1/10
1169/1169 [==============================] - 132s 111ms/step - loss: 14.2288 - MAE: 14.2288 - val_loss: 12.9011 - val_MAE: 12.9011
Epoch 2/10
 459/1169 [==========>...................] - ETA: 54s - loss: 13.8113 - MAE: 13.8113

## Visualizing the training process with tensorboard

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs

## Evaluating the model

In [ ]:
loss, accuracy = model.evaluate(test_ds)

print("Loss: ", loss)
print("Accuracy: ", accuracy)